practice

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import  CountVectorizer
from scipy import sparse
from sklearn import preprocessing, model_selection
from scipy.stats import boxcox
import string
%matplotlib inline

In [2]:
train_test = pd.read_json('../input/train.json')


In [3]:
train_test['interest_level'] = train_test['interest_level']
train_test['features2'] = train_test['features']
train_test['features2'] = train_test['features2'].apply(lambda x: ' '.join(x))

c_vect = CountVectorizer(stop_words='english', max_features=200, ngram_range=(1, 1))
c_vect.fit(train_test['features2'])

c_vect_sparse_1 = c_vect.transform(train_test['features2'])
c_vect_sparse1_cols = c_vect.get_feature_names()
countvec = CountVectorizer(vocabulary=c_vect_sparse1_cols)
# adds feature column for amount of "features" 
## train_test.drop(['features', 'features2'], axis=1, inplace=True)

## c_vect_sparse1_cols
dfinal = pd.DataFrame(countvec.fit_transform(train_test.features2).toarray(), 
                       index=train_test.features2, columns=countvec.get_feature_names())

dfinal.head()


interest_dummies = pd.get_dummies(dfinal.interest_level)
dfinal = pd.concat([dfinal,interest_dummies[['low','medium','high']]], axis = 1).drop('interest_level', axis = 1)

dfinal.head()

In [4]:
df_words2 = dfinal.sum(axis=0)
df_words2.head()

In [ ]:

df_words = dfinal.sum(axis=0).sort_values(ascending=False)
df_words = pd.DataFrame(df_words, columns=['Word_Count'])
df_words

# shows sum of words used in features

In [ ]:
ix20 = int(len(df_words['Word_Count'].dropna())*0.2)
print("TRAIN: 20% of words ({0}) responsible for {1:2.2f}% of entries".format(ix20,df_words['Word_Count'].sort_values(ascending = False).cumsum().iloc[ix20]/df_words['Word_Count'].sum()*100))

# exactly 20% of feature words are used in 95.54% of entries

In [ ]:
temp1 = train_test['interest_level']
temp2 = train_test['features2']
fwords = pd.concat([temp1,temp2], axis = 1, join = 'outer')
fwords.columns = ['interest_level','features2']
print(fwords.head(20))
